# Python Function Sample

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to develop a python function based model. This type of model is useful as user would be able to define their own logic inside the model as long as it satisfy contract given in `merlin.PyFuncModel`.
The model that we are going to develop is an ensembling of xgboost and sklearn model.

In [ ]:
!pip install --upgrade -r requirements.txt > /dev/null

In [6]:
import merlin
import warnings
import os
import xgboost as xgb
from merlin.model import ModelType, PyFuncModel
from sklearn import svm
from sklearn.datasets import load_iris
from joblib import dump
warnings.filterwarnings('ignore')

## 1. Initialize

### 1.1 Set Server

In [ ]:
# Set MLP Server
MERLIN_SERVER_URL = os.environ.get("MERLIN_SERVER_URL", "localhost:8080/api/merlin")
merlin.set_url(MERLIN_SERVER_URL)

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [ ]:
merlin.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in MLP is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by MLP are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
merlin.set_model("pyfunc-sample-2", ModelType.PYFUNC)

## 2. Train Model

In this step we are going to train 2 IRIS classifier model and combine the prediction result into a single model which will be implemented as a PyFunc type model.

### 2.1 Train First Model

In [7]:
model_1_dir = "xgboost-model"
BST_FILE = "model_1.bst"

iris = load_iris()
y = iris['target']
X = iris['data']
dtrain = xgb.DMatrix(X, label=y)
param = {'max_depth': 6,
            'eta': 0.1,
            'silent': 1,
            'nthread': 4,
            'num_class': 3,
            'objective': 'multi:softprob'
            }
xgb_model = xgb.train(params=param, dtrain=dtrain)
model_1_path = os.path.join(model_1_dir, BST_FILE)
xgb_model.save_model(model_1_path)

[12:10:08] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




### 2.2 Train Second Model

In [8]:
model_2_dir = "sklearn-model"
MODEL_FILE = "model_2.joblib"
model_2_path = os.path.join(model_2_dir, MODEL_FILE)

clf = svm.SVC(gamma='scale', probability=True)
clf.fit(X, y)
dump(clf, model_2_path)

['sklearn-model/model_2.joblib']

### 2.3 Create PyFunc Model

To create a PyFunc model you'll have to extend `merlin.PyFuncModel` class and implement its `initialize` and `infer` method.

`initialize` will be called once during model initialization. The argument to `initialize` is a dictionary containing a key value pair of artifact name and its URL. The artifact's keys are the same value as received by `log_pyfunc_model`.

`infer` method is the prediction method that is need to be implemented. It accept a dictionary type argument which represent incoming request body. `infer` should return a dictionary object which correspond to response body of prediction result.

In following example we are creating PyFunc model called `EnsembleModel`. In its `initialize` method we expect 2 artifacts called `xgb_model` and `sklearn_model`, those 2 artifacts would point to the serialized model file of each model. The `infer` method will simply does prediction for both model and return the average value.

In [9]:
import xgboost as xgb
import joblib
import numpy as np

class EnsembleModel(PyFuncModel):
    def initialize(self, artifacts):
        self._model_1 = xgb.Booster(model_file=artifacts["xgb_model"])
        self._model_2 = joblib.load(artifacts["sklearn_model"])
    
    def infer(self, request, **kwargs):
        model_input = request["instances"]
        inputs = np.array(model_input)
        dmatrix = xgb.DMatrix(inputs)
        result_1 = self._model_1.predict(dmatrix)
        result_2 = self._model_2.predict_proba(inputs)
        return {"predictions": ((result_1 + result_2) / 2).tolist()}

Let's test it locally

In [10]:
m = EnsembleModel()
m.initialize({"xgb_model": model_1_path, "sklearn_model": model_2_path})
m.infer({"instances": [[1,2,3,4], [2,1,2,4]] })

{'predictions': [[0.22366186074437142, 0.3047382124338789, 0.4715999044700078],
  [0.43930651004457333, 0.2117506019399637, 0.3489429066419144]]}

### 2.4 Run PyFunc Model Server Locally (Optional)

For faster development iteration, you can also simulate running the PyFunc model server in your local machine, by calling `merlin.run_pyfunc_model` function. `run_pyfunc_model` is a blocking function that will start a PyFunc model server in port 8080 by default.

As preqrequisites, you need to have Docker installed in your machine.

In [ ]:
merlin.run_pyfunc_model(
    model_instance=EnsembleModel(), 
    conda_env="env.yaml", 
    artifacts={"xgb_model": model_1_path, "sklearn_model": model_2_path},
)

2024/01/15 12:10:13 INFO mlflow.tracking.fluent: Experiment with name 'ensemblemodel' does not exist. Creating a new experiment.


Logging model to local MLflow
Building Docker image ensemblemodel-dev
Running PyFunc local server
b'/opt/conda/envs/merlin-model/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:\n'
b'https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations\n'
b'  warnings.warn(\n'
b'INFO:root:Registering model: name: ensemblemodel, version: dev, fullname: ensemblemodel-dev\n'
b'INFO:root:Listening on port 8080\n'
b'INFO:root:Will fork 1 workers\n'


Once the PyFunc model server is running, you will see a log like this:

```
Running PyFunc local server
b'INFO:root:Registering model: name: ensemblemodel, version: dev, fullname: ensemblemodel-dev\n'
b'INFO:root:Listening on port 8080\n'
b'INFO:root:Will fork 1 workers\n'
```

Now, you can send the request using curl from your terminal:

In [ ]:
%%bash
curl -X POST "http://localhost:8080/v1/models/ensemblemodel-dev:predict" -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

## 3. Deploy Model

To deploy the model, we will have to create an iteration of the model (by create a `model_version`), upload the serialized model to MLP, and then deploy.

### 3.1 Create Model Version and Upload

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_pyfunc_model(model_instance=EnsembleModel(), 
                conda_env="env.yaml", 
                artifacts={"xgb_model": model_1_path, "sklearn_model": model_2_path})
v.finish()
```

To upload PyFunc model you have to provide following arguments:
1. `model_instance` is the instance of PyFunc model, the model has to extend `merlin.PyFuncModel`
2. `conda_env` is path to conda environment yaml file. The environment yaml file must contain all dependency required by the PyFunc model.
3. (Optional) `artifacts` is additional artifact that you want to include in the model
4. (Optional) `code_dir` is a list of directory containing python code that will be loaded during model initialization, this is required when `model_instance` depend on local python package

In [ ]:
with merlin.new_model_version() as v:    
    merlin.log_pyfunc_model(
        model_instance=EnsembleModel(), 
        conda_env="env.yaml", 
        artifacts={"xgb_model": model_1_path, "sklearn_model": model_2_path},
    )

### 3.2 Deploy Model

We can also pass environment variable to the model during deployment by passing a dictionary of environment variables

In [ ]:
env_vars = {"WORKERS": "1"}

Each of a deployed model version will have its own generated url

In [ ]:
endpoint = merlin.deploy(v, env_vars=env_vars)

### 3.3 Send Test Request

In [ ]:
%%bash -s "$endpoint.url"
curl -v -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

### 3.4 Delete Deployment

In [ ]:
merlin.undeploy(v)

# 4. Testing PyFunc Model Server Locally

Up until this point, you have successfully built and run your PyFunc model on the Merlin server. To have faster development iteration, you can also simulate running the PyFunc model server in your local machine, by calling `merlin.run_pyfunc_model` function.

In [5]:
merlin.run_pyfunc_model(
    model_instance=EnsembleModel(), 
    conda_env="env.yaml", 
    artifacts={"xgb_model": model_1_path, "sklearn_model": model_2_path},
)

Logging model to local MLflow
Building Docker image ensemblemodel-dev
Running PyFunc local server
b'2024/01/09 06:58:58 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.13`, differs from the version of Python that is currently running, `Python 3.8.18`, and may be incompatible\n'
b'INFO:root:Registering model: name: ensemblemodel, version: dev, fullname: ensemblemodel-dev\n'
b'INFO:root:Listening on port 8080\n'
b'INFO:root:Will fork 1 workers\n'


KeyboardInterrupt: 

Once the PyFunc model server is running, you can send the requests using curl from your terminal:

```
curl -X POST "http://localhost:8080/v1/models/ensemblemodel-dev:predict" -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'
```